In [1]:
import pandas as pd
from torch_geometric.data import HeteroData
import torch

In [2]:
data = pd.read_csv('../Data/clean_edge.csv')
timestep_one = data[data['minutes'] == 1]
same_layer = timestep_one[timestep_one['ori_layer'] == timestep_one['des_layer']]

In [3]:
same_layer['node_type'] = same_layer['ori_layer']
same_layer.drop(columns=['ori_layer', 'des_layer', 'minutes'], inplace=True)

/var/folders/jr/kqd9xx3x423_tdgk_zbvkz0r0000gs/T/ipykernel_36947/1714444894.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  same_layer['node_type'] = same_layer['ori_layer']
/var/folders/jr/kqd9xx3x423_tdgk_zbvkz0r0000gs/T/ipykernel_36947/1714444894.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  same_layer.drop(columns=['ori_layer', 'des_layer', 'minutes'], inplace=True)


In [4]:
same_layer

,ori_node,des_node,km,ori_area,des_area,ori_node_latlong,des_node_latlong,node_type
25,2400,2400,0.000,National - National Ferry,National - National Ferry,"(51.20442, -0.20888)","(51.20442, -0.20888)",1
43,6626,6626,0.000,Western Isles,Western Isles,"(51.88044, 0.36945)","(51.88044, 0.36945)",1
55,7550,7550,0.000,National - National Ferry,National - National Ferry,"(51.26128, -0.58046)","(51.26128, -0.58046)",1
62,10034,10034,0.000,Argyll & Bute,Argyll & Bute,"(54.68133, -1.20555)","(54.68133, -1.20555)",1
74,12417,12417,0.000,National - National Ferry,National - National Ferry,"(56.19289, -3.05277)","(56.19289, -3.05277)",1
...,...,...,...,...,...,...,...,...
465211,262379,81120,0.201,Oxfordshire,Oxfordshire,"(52.41516, -1.81057)","(51.14423, 0.31447)",5
465212,262380,45790,0.998,Cheshire West & Chester,Cheshire West & Chester,"(53.39095, -2.97003)","(51.29202, 0.41866)",5
465213,262381,187358,0.446,Greater Manchester,Greater Manchester,"(53.39077, -2.97033)","(50.92588, -0.75803)",5
465214,262382,168519,0.313,Merseyside,Merseyside,"(51.31514, -2.20666)","(52.9154, -1.50813)",5


In [5]:
source_nodes = same_layer['ori_node']
dest_nodes = same_layer['des_node']

all_nodes = pd.concat([source_nodes, dest_nodes])
unique_nodes = all_nodes.unique()

num_unique_nodes = len(unique_nodes)

same_layer = same_layer[(same_layer['ori_node'] >= 0) & (same_layer['ori_node'] < num_unique_nodes) &
                 (same_layer['des_node'] >= 0) & (same_layer['des_node'] < num_unique_nodes)]

In [6]:
type_1 = same_layer[same_layer['node_type'] == 1]
type_2 = same_layer[same_layer['node_type'] == 2]
type_3 = same_layer[same_layer['node_type'] == 3]
type_4 = same_layer[same_layer['node_type'] == 4]
type_5 = same_layer[same_layer['node_type'] == 5]

In [7]:
edge_index_type1 = torch.tensor([type_1['ori_node'].values, type_1['des_node'].values], dtype=torch.long)
edge_index_type2 = torch.tensor([type_2['ori_node'].values, type_2['des_node'].values], dtype=torch.long)
edge_index_type3 = torch.tensor([type_3['ori_node'].values, type_3['des_node'].values], dtype=torch.long)
edge_index_type4 = torch.tensor([type_4['ori_node'].values, type_4['des_node'].values], dtype=torch.long)
edge_index_type5 = torch.tensor([type_5['ori_node'].values, type_5['des_node'].values], dtype=torch.long)

/var/folders/jr/kqd9xx3x423_tdgk_zbvkz0r0000gs/T/ipykernel_36947/2015740975.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:281.)
  edge_index_type1 = torch.tensor([type_1['ori_node'].values, type_1['des_node'].values], dtype=torch.long)


In [8]:
same_layer

,ori_node,des_node,km,ori_area,des_area,ori_node_latlong,des_node_latlong,node_type
25,2400,2400,0.000,National - National Ferry,National - National Ferry,"(51.20442, -0.20888)","(51.20442, -0.20888)",1
43,6626,6626,0.000,Western Isles,Western Isles,"(51.88044, 0.36945)","(51.88044, 0.36945)",1
55,7550,7550,0.000,National - National Ferry,National - National Ferry,"(51.26128, -0.58046)","(51.26128, -0.58046)",1
62,10034,10034,0.000,Argyll & Bute,Argyll & Bute,"(54.68133, -1.20555)","(54.68133, -1.20555)",1
74,12417,12417,0.000,National - National Ferry,National - National Ferry,"(56.19289, -3.05277)","(56.19289, -3.05277)",1
...,...,...,...,...,...,...,...,...
440091,244966,35000,0.301,Tyne & Wear,National - National Tram,"(53.07552, -2.23375)","(51.66697, -0.10946)",5
440092,244967,244961,0.258,Tyne & Wear,Tyne & Wear,"(53.05698, -2.17529)","(51.68182, -3.01207)",5
440093,244968,244970,0.317,Tyne & Wear,Tyne & Wear,"(51.70279, -1.78119)","(50.92857, -1.19878)",5
440094,244969,244967,0.143,Tyne & Wear,Tyne & Wear,"(52.48614, -1.48279)","(53.05698, -2.17529)",5


In [9]:
data = HeteroData()
data['node_type', 'edge_type1', 'node_type'].edge_index = edge_index_type1
data['node_type', 'edge_type2', 'node_type'].edge_index = edge_index_type2
data['node_type', 'edge_type3', 'node_type'].edge_index = edge_index_type3
data['node_type', 'edge_type4', 'node_type'].edge_index = edge_index_type4
data['node_type', 'edge_type5', 'node_type'].edge_index = edge_index_type5

In [10]:
source_nodes = same_layer['ori_node']
dest_nodes = same_layer['des_node']

all_nodes = pd.concat([source_nodes, dest_nodes])
num_unique_nodes = all_nodes.nunique()

In [11]:
import torch_geometric.transforms as T
data = T.ToUndirected()(data)
data['node_type'].num_nodes = num_unique_nodes
data['node_type'].node_id = torch.arange(num_unique_nodes)

In [12]:
edge_types = ['edge_type1', 'edge_type2', 'edge_type3', 'edge_type4', 'edge_type5']
for i in edge_types:
    edge_index = data['node_type', i, 'node_type'].edge_index
    valid_mask = (edge_index[0] < num_unique_nodes) & (edge_index[1] < num_unique_nodes)
    filtered_edge_index = edge_index[:, valid_mask]

    # Update the edge index in the HeteroData object
    data['node_type', i, 'node_type'].edge_index = filtered_edge_index

In [14]:
transform = T.RandomLinkSplit(
    num_val=0.1,
    num_test=0.1,
    disjoint_train_ratio=0.3,
    neg_sampling_ratio=2.0,
    add_negative_train_samples=False,
    edge_types=[
        ('node_type', 'edge_type1', 'node_type'),
        ('node_type', 'edge_type2', 'node_type'),
        ('node_type', 'edge_type3', 'node_type'),
        ('node_type', 'edge_type4', 'node_type'),
        ('node_type', 'edge_type5', 'node_type')
    ]
)
train_data, val_data, test_data = transform(data)

In [34]:
from torch_geometric.loader import LinkNeighborLoader

# Define seed edges:
edge_label_index = train_data['node_type', 'edge_type3', 'node_type'].edge_label_index
edge_label = train_data['node_type', 'edge_type3', 'node_type'].edge_label
train_loader = LinkNeighborLoader(
    data=train_data,
    num_neighbors=[20, 10],
    neg_sampling_ratio=2.0,
    edge_label_index=(("node_type", "edge_type3", "node_type"), edge_label_index),
    edge_label=edge_label,
    batch_size=32,
    shuffle=True,
)

edge_label_index = val_data["node_type", "edge_type3", "node_type"].edge_label_index
edge_label = val_data["node_type", "edge_type3", "node_type"].edge_label
val_loader = LinkNeighborLoader(
    data=val_data,
    num_neighbors=[20, 10],
    edge_label_index=(("node_type", "edge_type3", "node_type"), edge_label_index),
    edge_label=edge_label,
    batch_size=3 * 128,
    shuffle=False,
    )

In [43]:
from torch_geometric.nn import SAGEConv, to_hetero
import torch.nn.functional as F
import tqdm
from sklearn.metrics import roc_auc_score, accuracy_score

class GNN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.conv1 = SAGEConv(hidden_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)

    def forward(self, x, edge_index):

        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x

class Classifier(torch.nn.Module):
    def forward(self, x_user, edge_label_index):

        edge_feat_node1 = x_user[edge_label_index[0]]
        edge_feat_node2 = x_user[edge_label_index[1]]

        return (edge_feat_node1 * edge_feat_node2).sum(dim=-1)

class Model(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()

        self.node_emb = torch.nn.Embedding(data["node_type"].num_nodes, hidden_channels)

        self.gnn = GNN(hidden_channels)
        self.gnn = to_hetero(self.gnn, metadata=data.metadata())

        self.classifier = Classifier()

    def forward(self, data: HeteroData):
        x_dict = {
          "node_type": self.node_emb(data["node_type"].node_id)
          }
        
        
        x_dict = self.gnn(x_dict, data.edge_index_dict)
        pred = self.classifier(
            x_dict["node_type"],
            data["node_type", "edge_type3", "node_type"].edge_label_index,
        )
        return pred
        
model = Model(hidden_channels=64)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
for epoch in range(0, 151):
    total_loss = total_examples = total_correct = 0
    val_loss = val_examples = val_correct = 0
    for sampled_data in train_loader:
        optimizer.zero_grad()
        sampled_data.to(device)
        pred = model(sampled_data)
        ground_truth = sampled_data["node_type", "edge_type3", "node_type"].edge_label
        loss = F.binary_cross_entropy_with_logits(pred, ground_truth)
        loss.backward()
        optimizer.step()

        total_loss += float(loss) * pred.numel()
        total_examples += pred.numel()

    preds = []
    ground_truths = []
    for sampled_data in val_loader:
        with torch.no_grad():
            sampled_data.to(device)
            pred = model(sampled_data)
            ground_truth = sampled_data["node_type", "edge_type3", "node_type"].edge_label

            preds.append(pred)
            ground_truths.append(ground_truth)

            loss = F.binary_cross_entropy_with_logits(pred, ground_truth)
            val_loss += float(loss) * pred.numel()
            val_examples += pred.numel()

    val_pred = torch.cat(preds, dim=0).cpu().numpy()
    val_ground_truth = torch.cat(ground_truths, dim=0).cpu().numpy()

    auc = roc_auc_score(val_ground_truth, val_pred)
    val_pred_labels = (val_pred > 0.5).astype(int)
    accuracy = accuracy_score(val_ground_truth, val_pred_labels)

    if epoch % 10 == 0:
        print(f"Epoch: {epoch}, Val Loss: {val_loss / val_examples:.4f}, Validation AUC: {auc:.4f}, Validation Accuracy: {accuracy:.4f}")

Epoch: 0, Val Loss: 6.4114, Validation AUC: 0.6330, Validation Accuracy: 0.4821
Epoch: 10, Val Loss: 2.6100, Validation AUC: 0.7396, Validation Accuracy: 0.5714
Epoch: 20, Val Loss: 1.8595, Validation AUC: 0.7583, Validation Accuracy: 0.6250
Epoch: 30, Val Loss: 1.2649, Validation AUC: 0.7765, Validation Accuracy: 0.6905
Epoch: 40, Val Loss: 1.0652, Validation AUC: 0.7822, Validation Accuracy: 0.6667
Epoch: 50, Val Loss: 0.9513, Validation AUC: 0.7871, Validation Accuracy: 0.6667
Epoch: 60, Val Loss: 0.8738, Validation AUC: 0.7969, Validation Accuracy: 0.6667
Epoch: 70, Val Loss: 0.7670, Validation AUC: 0.8267, Validation Accuracy: 0.7143
Epoch: 80, Val Loss: 0.7267, Validation AUC: 0.8471, Validation Accuracy: 0.7262
Epoch: 90, Val Loss: 0.6853, Validation AUC: 0.8608, Validation Accuracy: 0.7500
Epoch: 100, Val Loss: 0.6580, Validation AUC: 0.8535, Validation Accuracy: 0.7560
Epoch: 110, Val Loss: 0.6561, Validation AUC: 0.8509, Validation Accuracy: 0.7679
Epoch: 120, Val Loss: 0.621